### Importing libraries:

In [ ]:
from stravalib.client import Client
import matplotlib.pyplot as plt
from pprint import pprint
from flask import Flask
import seaborn as sns
import pandas as pd
import requests
import numpy
import json

### Loading Functions:

In [ ]:
def compile_activities(club_activities, export=False):
    """Compile all club activities data to a dictionary (created previously or loaded from a file)"""

    # Acessing external JSON file with users data:
    with open('../users_data.json', 'r') as file:
        user_dict = json.load(file)

    for activity in club_activities:
        # Loads activity dictionary:
        act_dict = activity.to_dict()
        # Loads interest variables:
        user_name = "{} {}".format(act_dict['athlete']['firstname'], act_dict['athlete']['lastname'])
        val_distance = act_dict['distance']
        val_elapsed_time = time_converter(act_dict['elapsed_time'])
        val_moving_time = time_converter(act_dict['moving_time'])
        val_elevation_gain = act_dict['total_elevation_gain']
        # Loads user data whose match with the activity:
        user_data = user_dict[user_name]
        # Updating variables:
        user_data['activities_counter'] += 1
        user_data['distance'] += int(val_distance)
        user_data['total_elapsed_time'] += val_elapsed_time
        user_data['total_moving_time'] += val_moving_time
        # Compares if the value is bigger than the previous (and saves if it is):
        if user_data['highest_elevation_gain'] < val_elevation_gain:
            user_data['highest_elevation_gain'] = val_elevation_gain
        #Displays progress on console when export is enable:
        if export == True:
            print("Activity saved! User: {}".format(user_name))

    # The dict is displayed on the console:
    if export == False:
        return user_dict
    # The dict is exported as a JSON file:
    elif export == True:
        with open('../users_data_compiled.json', 'w') as users_file:
            json.dump(user_dict, users_file)
            
            
def load_file_data(filename, display=False):
    """Display data in the file"""

    # Loads file and read it's data:
    with open('../{}'.format(filename), 'r') as file:
        data = json.load(file)
    # Shows data in the console:
    if display == True:
        pprint(data)
    elif display == False:
        return data
    
    
def new_users_file(users_list, export=False):
    """Creates a dictionary to store data about each user considering summary-level data acess"""

    # Creates empty dictionary:
    users_dict = {}
    for user in users_list:
        # Load user fullname to be used as dictionary key:
        user_fullname = user.to_dict()['firstname'] + " " + user.to_dict()['lastname']
        # Check if the file already contains the user data:
        try:
            users_dict[user_fullname]
        # If there's no user data it creates a new one:
        except :
            users_dict[user_fullname] = {'activities_counter': 0,
                                         'distance': 0,
                                         'total_elapsed_time': 0,
                                         'total_moving_time': 0,
                                         'highest_elevation_gain': 0}

    # The dict is displayed on the console:
    if export == False:
        return users_dict
    # The dict is exported as a JSON file:
    elif export == True:
        with open('../users_data.json', 'w') as users_file:
            json.dump(users_dict, users_file)


def time_converter(time_value):
    """Converts a string variable in the format HH:MM:SS to a int variable in seconds, and vice-versa"""

    # Convertion string to integer:
    if type(time_value) == str:
        values = time_value.split(":")
        hours = int(values[0])
        minutes = int(values[1])
        seconds = int(values[2])
        total_time = seconds + minutes*60 + hours*3600

        return total_time

    # Converts integer to string:
    elif type(time_value) == int:
        hours = 0
        minutes = 0 
        seconds = 0
        while time_value != 0:
            if time_value >= 3600:
                hours += 1
                time_value -= 3600
            elif time_value >= 60:
                minutes += 1
                time_value -= 60
            else:
                seconds += 1
                time_value -= 1

        return "{}:{}:{}".format(hours, minutes, seconds)


    else:
        return "Invalid data type\nVariable type must be 'str' or 'int'"

### Load Strava Client and Interest Variables:

In [ ]:
# Acessing external JSON file with developer data acess:
with open('../dev_log.json', 'r') as file:
    dev_log = json.load(file)

# Defining variables:
client_ID = dev_log['client_ID']
client_secret = dev_log['client_secret']
acess_token = dev_log['acess_token']
club_ID = dev_log['club_ID']


# Creating developer-level acess to Strava's API:
strava_client = Client(acess_token)

# Acessing developer Strava data (Used to prepare program to 'resource_state=3' (Detail-level data)):
dev_user = strava_client.get_athlete().to_dict()
dev_act = list(strava_client.get_activities())

# Acessing Club Data:
club = strava_client.get_club(club_ID).to_dict()

# Acessing Club Members Data:
club_users = list(strava_client.get_club_members(club_ID))

# Acessing Club Activities Data:
club_act = list(strava_client.get_club_activities(club_ID))


## Debug Console:

In [ ]:
#print(club_users[0])
#pprint(club_users[0].to_dict())
#print(len(club_act))
#pprint(club_act[0].to_dict())

#new_users_file(club_users, export=True)
pprint(compile_activities(club_act, export=True))
#pprint(dev_user)
#pprint(dev_act[0].to_dict())


In [ ]:
pprint(club_act[-1].to_dict())

In [ ]:
print(len(club_act))

### Data Filtering and Analysis:

In [ ]:
df = pd.DataFrame(load_file_data('users_data_compiled.json')).transpose()
df

In [ ]:
df_clean = df[(df.activities_counter != 0) & 
              (df.distance != 0) & 
              (df.highest_elevation_gain != 0) & 
              (df.total_elapsed_time != 0) & 
              (df.total_moving_time != 0)]
df_clean

### Graphic Analysis:

In [ ]:
# Setting Seaborn configs:
sns.set()

In [ ]:

plt.bar(list(range(df_clean.shape[0])), df_clean['activities_counter'])
plt.xticks(list(range(df_clean.shape[0])), df_clean.index, rotation='vertical')
plt.xlabel('Users')
plt.ylabel('Amount')
plt.show()

plt.bar(list(range(df_clean.shape[0])), df_clean['distance'])
plt.xticks(list(range(df_clean.shape[0])), df_clean.index, rotation='vertical')
plt.xlabel('Users')
plt.ylabel('Amount')
plt.show()

### Web Server:

In [ ]:
redirect_uri = "http://127.0.0.1:5000/authorize"
auth_url = strava_client.authorization_url(client_ID, redirect_uri)

# Flask web server to create acess to user authorization:
def server_on(debug=False):
    app = Flask(__name__)

    @app.route("/")
    def root():
        return "Hello World!"

    @app.route("/authorize")
    def authorize():
        from flask import redirect
        return redirect(auth_url)
    
    @app.route("/authorization_successful")
    def authorization_successful():
        code = requests.get(auth_url).code
        user_token = strava_client.exchange_code_for_token(client_ID, client_secret, code)
        
    @app.errorhandler(404)
    def page_not_found(error):
        from flask import render_template
        return render_template('page_not_found.html'), 404

    # Execute web server (localhost):
    if debug == False:
        app.run("http://127.0.0.1:5000"")
    elif debug == True:
        app.run(debug=True)

In [ ]:
server_on()

In [ ]:
print(auth_url)